# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Ejemplos de Spark: Structured Streaming (Files)** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("tarea-Files1") \
    .master("spark://fc91669459e3:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/02 17:33:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from whatsapp2.spark_utils import SparkUtils
from whatsapp2.logs_query_listener import LogsListener

spark.streams.addListener(LogsListener())

### Creación del FileStream

In [4]:
from team_name.spark_utils import SparkUtils
from pyspark.sql.functions import split


logs_df = spark \
                .readStream \
                .format("text") \
                .option("maxFilesPerTrigger", 1) \
                .load("/home/jovyan/notebooks/data/structured_streaming_files/logs/")



In [5]:
structured_logs_df = logs_df.select(split(logs_df.value, " \\| ").alias("log"))
structured_logs_df = structured_logs_df.withColumn("timestamp", structured_logs_df["log"].getItem(0).cast("timestamp"))
structured_logs_df = structured_logs_df.withColumn("level", structured_logs_df["log"].getItem(1).cast("string"))
structured_logs_df = structured_logs_df.withColumn("message", structured_logs_df["log"].getItem(2).cast("string"))
structured_logs_df = structured_logs_df.withColumn("server", structured_logs_df["log"].getItem(3).cast("string"))

structured_logs_df.printSchema()

root
 |-- log: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- timestamp: timestamp (nullable = true)
 |-- level: string (nullable = true)
 |-- message: string (nullable = true)
 |-- server: string (nullable = true)



In [6]:
structured_logs_df = structured_logs_df.filter(structured_logs_df["level"] == "ERROR")

### Configuración del "Sink" del stream

In [ ]:
query = structured_logs_df \
                .writeStream \
                .outputMode("append") \
                .trigger(processingTime='10 seconds') \
                .format("console") \
                .start()

query.awaitTermination(40)


In [ ]:
query.stop()

In [ ]:
sc.stop()